# Lab 8 - Chain-of-Thought OpenAI GPT-4o-mini

In [ ]:
require 'json'
require 'ruby-openai'
require 'dotenv'

# Load environment variables from .env file
Dotenv.load

# Don't include keys like this, use ENV vars!
config = JSON.parse(File.read('config.json'))
client = OpenAI::Client.new(access_token: config['openai_key'])

In [ ]:
def complete(prompt)
  response = client.chat(
    parameters: {
      model: "gpt-4o-mini",
      messages: [{ role: "user", content: prompt }],
      temperature: 0
    }
  )

  response.dig("choices", 0, "message", "content")
end

In [ ]:
def get_steps_prompt(query)
  "#{query}\n\nBefore answering, think step-by-step."
end

In [ ]:
def get_answer_prompt(query, steps)
  "#{query}\n\n#{steps}\n\nANSWER:"
end

In [ ]:
def answer(query)
  # Get the first prompt, including the step-by-step instruction
  prompt_1 = get_steps_prompt(query)
  
  # We get the chain-of-thought steps response back from GPT
  puts "Thinking..."
  steps = complete(prompt_1)
  puts "Steps generated"
  
  # Get the answer portion of the chain-of-thought-prompt
  puts "Thinking..."
  prompt_2 = get_answer_prompt(query, steps)
  puts "Answer extracted"

  response = complete(prompt_2)
  puts response
  
  response
end

In [ ]:
# Ask a question
answer("Alice and Bob go to a hotel with 255 rooms. Bob sleeps in one of the rooms and Alice sleeps in another. How many vacant rooms are available?")

## Using OpenAI structured output for a single-request Chain-of-Thought answer

In [ ]:
def answer_chain_of_thought(name, prompt)
  puts "Thinking..."
  
  schema = {
    name: name,
    strict: true,
    schema: {
      type: "object",
      properties: {
        steps: {
          type: "array",
          items: {
            type: "object",
            properties: {
              explanation: { type: "string" },
              output: { type: "string" }
            },
            required: ["explanation", "output"],
            additionalProperties: false
          }
        },
        final_answer: { type: "string" }
      },
      required: ["steps", "final_answer"],
      additionalProperties: false
    }
  }

  response = client.chat(
    parameters: {
      model: "gpt-4o-mini",
      messages: [
        {
          role: "system",
          content: "You are a helpful assistant. Always analyze and think step-by-step before responding with the final answer."
        },
        {
          role: "user",
          content: prompt
        }
      ],
      temperature: 0,
      response_format: {
        type: "json_schema",
        json_schema: schema
      }
    }
  )

  begin
    content = response.dig("choices", 0, "message", "content")
    content_json = JSON.parse(content)
    steps = content_json["steps"]
    final_answer = content_json["final_answer"]
    
    puts "Steps:"
    puts steps
    puts "Final Answer:"
    puts final_answer
    
    final_answer
  rescue StandardError => e
    puts response
    puts e.message
    nil
  end
end

In [ ]:
# Ask a question
answer_chain_of_thought("vacancy_calculator", "Alice and Bob go to a hotel with 255 rooms. Bob sleeps in one of the rooms and Alice sleeps in another. How many vacant rooms are available?")